In [ ]:
import gdown

url = "https://drive.google.com/uc?id=1XSw0lp2UlbnCJ5qrV1ORtoEJ-lBCnvAF"

output = "data_file.zip"

gdown.download(url, output, quiet=False)

print(f"Файл сохранён как {output}")


Downloading...
From (original): https://drive.google.com/uc?id=1XSw0lp2UlbnCJ5qrV1ORtoEJ-lBCnvAF
From (redirected): https://drive.google.com/uc?id=1XSw0lp2UlbnCJ5qrV1ORtoEJ-lBCnvAF&confirm=t&uuid=1084e333-b76d-47bc-843f-805ab341c86c
To: /content/data_file.zip
100%|██████████| 75.5M/75.5M [00:01<00:00, 46.7MB/s]

Файл сохранён как data_file.zip


In [ ]:
!unzip -q data_file.zip

In [ ]:
!pip -q install torchdiffeq torch_geometric

  Preparing metadata (setup.py) ... done
ERROR: Operation cancelled by user


In [ ]:
import torch
import torch.nn as nn
from torchdiffeq import odeint_adjoint as odeint
import torch_geometric.nn as gnn
import torch_geometric as torchg


class GRANDDiffusionODEFunc(gnn.MessagePassing):
    def __init__(self, vert_dim, model_dim, num_heads):
        super().__init__(flow="target_to_source")

        self.model_dim = model_dim
        self.vert_dim = vert_dim
        self.num_heads = num_heads

        self.k = nn.Linear(vert_dim, num_heads * model_dim)
        self.q = nn.Linear(vert_dim, num_heads * model_dim)

    def update(self, aggr_out):
        return aggr_out

    def message(self, x_j, x_i, index):
        Q_i = self.q(x_i).view(-1, self.num_heads, self.model_dim)
        K_j = self.k(x_j).view(-1, self.num_heads, self.model_dim)

        scores = (Q_i * K_j).sum(dim=-1) / torch.sqrt(self.model_dim)
        scores = scores.mean(dim=1)
        alpha = torchg.utils.softmax(scores, index)
        return (x_j - x_i) * alpha.unsqueeze(1)

    def forward(self, edge_index, x):
        return self.propagate(edge_index, x=x)


class ODEFunc(nn.Module):
    def __init__(self, func, edge_index):
        super().__init__()
        self.func = func
        self.edge_index = edge_index

    def forward(self, t, x):
        return self.func(self.edge_index, x)


class GRANDAutoencoder(nn.Module):
    def __init__(self, edge_index, vert_dim, model_dim, num_heads):
        super().__init__()
        self.encoder = GRANDDiffusionODEFunc(vert_dim, model_dim, num_heads)
        self.decoder = GRANDDiffusionODEFunc(vert_dim, model_dim, num_heads)

        self.encoder_ode = ODEFunc(self.encoder, edge_index)
        self.decoder_ode = ODEFunc(self.decoder, edge_index)

    def forward(self, x, t):

        z = odeint(self.encoder_ode, x, t)[-1]
        x_hat = odeint(self.decoder_ode, z, t)[-1]

        return x_hat, z

In [ ]:
import polars as pl
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

from scipy.ndimage import gaussian_filter1d
from sklearn.preprocessing import StandardScaler

from scipy.spatial.distance import pdist, squareform
from scipy.sparse import csr_matrix
import plotly.graph_objects as go


data = pl.read_csv("A_DeviceMotion_data/A_DeviceMotion_data/wlk_7/sub_20.csv")

gyro_cols = ["rotationRate.x", "rotationRate.y", "rotationRate.z"]
acc_cols = ["userAcceleration.x", "userAcceleration.y", "userAcceleration.z"]

gyro = np.vstack([data[col].to_numpy() for col in gyro_cols]).T
acc = np.vstack([data[col].to_numpy() for col in acc_cols]).T

sigma_smooth = 5
gyro_smooth = gaussian_filter1d(gyro, sigma=sigma_smooth, axis=0)
acc_smooth = gaussian_filter1d(acc, sigma=sigma_smooth, axis=0)

heuristic_raw = np.sum(acc_smooth**2, axis=1)
heuristic_smooth = gaussian_filter1d(heuristic_raw, sigma=sigma_smooth)

scaler_gyro = StandardScaler()
gyro_scaled = scaler_gyro.fit_transform(gyro_smooth)

scaler_h = StandardScaler()
heuristic_scaled = scaler_h.fit_transform(heuristic_smooth.reshape(-1, 1))

embed_input = np.hstack([heuristic_scaled, gyro_scaled])


def embedd(signal, m, tau):
    N = len(signal)
    emb_len = N - tau * (m - 1)
    if emb_len <= 0:
        raise ValueError("m и tau слишком велики для длины сигнала")
    emb = np.zeros((emb_len, m * signal.shape[1]))
    for i in range(emb_len):
        window = signal[i : i + tau * (m - 1) + 1 : tau]
        emb[i] = window.flatten()
    return emb


m, tau = 15, 4
X = embedd(embed_input, m=m, tau=tau)[:1000]


def build_gaussian_graph(X, sigma=None, epsilon=0.1, symmetric=True):
    N = X.shape[0]
    if sigma is None:
        sigma = np.median(pdist(X))
        print(f"Автоматически выбран sigma = {sigma:.4f}")

    dist_sq = squareform(pdist(X, metric="sqeuclidean"))  # [N, N]

    W = np.exp(-dist_sq / (2 * sigma**2))
    np.fill_diagonal(W, 0.0)

    W[W < epsilon] = 0.0

    if symmetric:
        W = np.maximum(W, W.T)

    sparse_W = csr_matrix(W)
    rows, cols = sparse_W.nonzero()
    edge_index = np.vstack([rows, cols])
    edge_weight = sparse_W.data

    return edge_index, edge_weight, sigma


edge_index_np, edge_weight_np, sigma_used = build_gaussian_graph(
    X, sigma=None, epsilon=0.3
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

x = torch.from_numpy(X.astype(np.float32)).to(device)
edge_index = torch.from_numpy(edge_index_np.astype(np.int64)).to(device)
edge_weight = torch.from_numpy(edge_weight_np.astype(np.float32)).to(device)

In [ ]:
X.shape

(1000, 60)

In [ ]:
from tqdm import tqdm

vert_dim = x.shape[1]
model_dim = 64
num_heads = 4
t = torch.linspace(0, 0.1, 10).to(device)

model = GRANDAutoencoder(edge_index, vert_dim, model_dim, num_heads).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

epochs = 12

x_tensor = torch.from_numpy(X.astype(np.float32)).to(device)

for epoch in tqdm(range(1, epochs + 1)):
    optimizer.zero_grad()

    x_hat, z = model(x_tensor, t)

    loss = loss_fn(x_hat, x_tensor)

    loss.backward()

    optimizer.step()

    if epoch % 10 == 0:
        grad_means = [
            p.grad.abs().mean().item() for p in model.parameters() if p.grad is not None
        ]
        print(
            f"Epoch {epoch:03d} | Loss: {loss.item():.6f} | Grad mean: {sum(grad_means)/len(grad_means):.6f}"
        )
    else:
        print(f"Epoch {epoch:03d} | Loss: {loss.item():.6f}")

  8%|▊         | 1/12 [00:24<04:33, 24.83s/it]

Epoch 001 | Loss: 0.025829


 17%|█▋        | 2/12 [00:49<04:05, 24.57s/it]

Epoch 002 | Loss: 0.023165


 25%|██▌       | 3/12 [01:13<03:40, 24.47s/it]

...

Epoch 010 | Loss: 0.008536 | Grad mean: 0.000009


 92%|█████████▏| 11/12 [04:28<00:24, 24.33s/it]

Epoch 011 | Loss: 0.007737


100%|██████████| 12/12 [04:52<00:00, 24.38s/it]

Epoch 012 | Loss: 0.007097


In [ ]:
U, S, Vh = torch.linalg.svd(z, full_matrices=False)
z_low = U[:, :3] * S[:3]
Z_hat = z_low @ Vh[:3, :]

reconstruction_loss = torch.mean((z - Z_hat) ** 2)
print(reconstruction_loss.item())

z_low_np = z_low.detach().cpu().numpy()  # [N, 3]

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=z_low_np[:, 0],
            y=z_low_np[:, 1],
            z=z_low_np[:, 2],
            mode="lines+markers",
            marker=dict(
                size=3, color=np.arange(z_low_np.shape[0]), colorscale="Viridis"
            ),
            line=dict(color="blue", width=2),
        )
    ]
)

fig.update_layout(
    title="3D Latent Trajectory from GRAND Autoencoder",
    scene=dict(xaxis_title="z1", yaxis_title="z2", zaxis_title="z3"),
)
fig.show()

0.1465887725353241


In [ ]:
U, S, Vh = np.linalg.svd(X, full_matrices=False)
X_lowwww = U[:, :3] * S[:3]

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=X_lowwww[:, 0],
            y=X_lowwww[:, 1],
            z=X_lowwww[:, 2],
            mode="lines+markers",
            marker=dict(
                size=3, color=np.arange(X_lowwww.shape[0]), colorscale="Viridis"
            ),
            line=dict(color="blue", width=2),
        )
    ]
)

fig.update_layout(
    title="3D Latent Trajectory from GRAND Autoencoder",
    scene=dict(xaxis_title="z1", yaxis_title="z2", zaxis_title="z3"),
)
fig.show()

In [ ]:
U, S, Vh = np.linalg.svd(X, full_matrices=False)
X_low = U[:, :3] * S[:3]
X_hat = X_low @ Vh[:3, :]
reconstruction_loss = np.mean((X - X_hat) ** 2)
print("Reconstruction MSE:", reconstruction_loss)

Reconstruction MSE: 0.17484372879735985
